# How does LaminDB's `select` relate to SQLModel's `select`?

In [ ]:
import lamindb as ln
import sqlmodel as sqm
import lnschema_core as core
from datetime import datetime
import pandas as pd

ln.nb.header()

In [ ]:
engine = ln.settings.instance.db_engine()

In [ ]:
def to_df(select_list):
    return pd.DataFrame(
        [row.dict() for row in select_list], columns=select_list[0].__fields__
    )

## Selecting a whole table

### (A) SQLModel

In [ ]:
with sqm.Session(engine) as session:
    df = to_df(session.exec(sqm.select(core.dobject)).all())

In [ ]:
df

### (B) LaminDB

In [ ]:
df = ln.db.select.dobject().df()

In [ ]:
df

## Subsetting a selection with `where`: equality

### (A) SQLModel

In [ ]:
with sqm.Session(engine) as session:
    df = to_df(
        session.exec(
            sqm.select(core.dobject).where(core.dobject.suffix == ".feather")
        ).all()
    )

In [ ]:
df

### (B) LaminDB

In [ ]:
ln.db.select.dobject(suffix=".feather").df()

## Queries involving linked tables

### (A) SQLModel

In [ ]:
ln.db.select.user().df()

In [ ]:
user_id = "DzTjkKse"

Select version without joins, but using the `in_` operator.

In [ ]:
with sqm.Session(engine) as session:
    # all notebooks authored by the user
    jupynbs = session.exec(
        sqm.select(core.jupynb).where(core.jupynb.created_by == user_id)
    ).all()
    jupynb_ids = [jupynb.id for jupynb in jupynbs]
    # all dtransforms linked to these notebooks
    dtransforms = session.exec(
        sqm.select(core.dtransform).where(core.dtransform.jupynb_id.in_(jupynb_ids))
    ).all()
    dtransform_ids = [dtransform.id for dtransform in dtransforms]
    # all dobjects linked to these dtransforms
    df = to_df(
        session.exec(
            sqm.select(core.dobject).where(
                core.dobject.dtransform_id.in_(dtransform_ids)
            )
        ).all()
    )

In [ ]:
df

### LaminDB

In [ ]:
# ln.db.select.dobject(where=dict(user=dict(id="DzTjkKse")))

## Subsetting a selection with `where`: general comparisons

### (A) SQLModel

Subsetting the previous select to the most recent additions.

In [ ]:
time = datetime(2022, 10, 11, 23, 0, 0)

In [ ]:
with sqm.Session(engine) as session:
    df = to_df(
        session.exec(
            sqm.select(core.dobject).where(
                core.dobject.suffix == ".feather", core.dobject.created_at > time
            )
        ).all()
    )

In [ ]:
df

### (B) LaminDB

We can't do this right now.

## Subsetting a selection with `where`: combine expressions using `or`

SQLModel only.

## Subsetting a selection with `where`: limit select results

SQLModel only.

## Limiting and offsetting select results

This is important for selecting tables with very high numbers of rows.

SQLModel only.